In [3]:
import numpy as np
from soliket.ccl import CCL*
from cobaya.model import get_model
from cobaya.likelihood import Likelihood
import pyccl as ccl

SyntaxError: invalid syntax (<ipython-input-3-e7451beb3f9d>, line 2)

In [2]:
class Tester(Likelihood):
    ell_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/ell.npy"
    cl_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/cls.npy"
    dcl_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/dcls.npy"
    #params = {'b_hydro': {"prior": {"min": 0, "max": 1}}}
    def initialize(self):
        self.cl_data = np.load(self.cl_file)
        self.dcl_data = np.load(self.dcl_file)
        self.ell_data = np.load(self.ell_file)

    def get_requirements(self):
        return {'CCL': {"methods": {'theory': self._get_theory},
                        "kmax": 10,
                        "nonlinear": True}}

    def _get_theory(self, cosmo):
        tracer1 = ccl.CMBLensingTracer(cosmo, z_source = 1060)
        tracer2 = ccl.CMBLensingTracer(cosmo, z_source = 1060)
        #ell = np.logspace(np.log10(3), 3)
        cls = ccl.cls.angular_cl(cosmo, tracer1, tracer2, self.ell_data)
        return cls

    def logp(self, **pars):
        results = self.provider.get_CCL()
        cls = results['theory']
        r = cls - self.cl_data
        #print(t, self.cl_data)
        chi2 = r**2/self.dcl_data**2.
        #chi2 = np.dot(r, self.invcov.dot(r))
        return -0.5 * np.sum(chi2)

In [6]:
cosmo_params = {
    "Omega_c": 0.25,
    "Omega_b": 0.05,
    "h": 0.67,
    "n_s": 0.96
}

info = {"params": {#"omch2": cosmo_params['Omega_c'] * cosmo_params['h'] ** 2.,
                   "ombh2": cosmo_params['Omega_b'] * cosmo_params['h'] ** 2.,
                   "H0": cosmo_params['h'] * 100,
                   "ns": cosmo_params['n_s'],
                   "logA": {'prior':{'min': 1.61, 'max': 3.91}, 'latex': '\log(10^{10} A_\mathrm{s})', 'drop': True},
                   "As": {'value':lambda logA: 1e-10*np.exp(logA), 'latex': 'A_\mathrm{s}'},
                   "omegab": {'value':0.04914741792, 'latex': '\Omega_\mathrm{b}', 'drop': True},
                   "omegam": {'prior':{'min': 0.1, 'max': 0.9}, 'latex': '\Omega_\mathrm{m}', 'drop': True},
                   "omch2": {'value':lambda omegam, omegab, mnu, H0: (omegam-omegab)*(H0/100)**2-(mnu*(3.046/3)**0.75)/94.0708, 'latex': '\Omega_\mathrm{c} h^2'},                    
                   "tau": 0.6,
                   "mnu": 0.06},
        "likelihood": {"Tester": Tester},
        "theory": {
            "camb": None,
            "ccl": {"external": CCL, "nonlinear": False}
        },
        "debug": False, "stop_at_error": True, 
       "sampler": {"mcmc": {'max_tries': 1000}}}

In [7]:
from cobaya.run import run

updated_info, sampler = run(info)

[camb] Importing *auto-installed* CAMB (but defaulting to *global*).
[camb] Initialized!
[mcmc] Getting initial point... (this may take a few seconds)
[prior] Reference values or pdf's for some parameters were not provided. Sampling from the prior instead for those parameters.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {Tester: 20200.0, camb.transfers: 2.57, camb: 1.29, ccl: 96.6}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Initial point: logA:2.256268, omegam:0.32506
[mcmc] Sampling!
[mcmc] Progress @ 2021-03-11 15:16:17 : 1 steps taken, and 0 accepted.


KeyboardInterrupt: 